In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df


,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,3,25,2024-07-02 15:47:20
1,大安區,21,4,17,2024-07-02 15:46:20
2,大安區,16,4,12,2024-07-02 14:01:23
3,大安區,11,4,7,2024-07-02 15:38:19
4,大安區,16,3,13,2024-07-02 15:22:20
...,...,...,...,...,...
1425,臺大公館校區,30,15,3,2024-07-02 15:47:19
1426,臺大公館校區,20,19,1,2024-07-02 15:39:14
1427,臺大公館校區,24,12,4,2024-07-02 15:47:20
1428,臺大公館校區,40,31,0,2024-07-02 15:42:15


In [2]:
#使用指定軸上的一項或多項操作進行計算。
df1 = df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數','sum'),('站點數','count')])
df1

總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3819  156  1666  156  2114  156
中正區     3209  121  1603  121  1540  121
信義區     3331  106  1375  106  1842  106
內湖區     3499  168  1544  168  1902  168
北投區     2472   94   864   94  1558   94
南港區     2317   92   873   92  1435   92
士林區     2962  128  1318  128  1592  128
大同區     1630   62   535   62  1085   62
大安區     4987  185  2187  185  2742  185
文山區     2203   97   678   97  1525   97
松山區     2343   88   959   88  1321   88
臺大公館校區  1437   53   724   53   529   53
萬華區     2057   80   810   80  1235   80

In [6]:
#pivot
#stack,unstack
#欄位有名稱
s1 = df1.stack(level=[0,1],future_stack=True) #現在是series 
s1

行政區域           
中山區   總數量   車輛數    3819
            站點數     156
      可借數量  車輛數    1666
            站點數     156
      可還數量  車輛數    2114
                   ... 
萬華區   總數量   站點數      80
      可借數量  車輛數     810
            站點數      80
      可還數量  車輛數    1235
            站點數      80
Length: 78, dtype: int64

In [8]:
s1.index.names = ['行政區域','主題','數量']  #先給他名稱
s1

行政區域  主題    數量 
中山區   總數量   車輛數    3819
            站點數     156
      可借數量  車輛數    1666
            站點數     156
      可還數量  車輛數    2114
                   ... 
萬華區   總數量   站點數      80
      可借數量  車輛數     810
            站點數      80
      可還數量  車輛數    1235
            站點數      80
Length: 78, dtype: int64

In [10]:
s1.unstack(level=['數量'])

數量            車輛數  站點數
行政區域   主題             
中山區    總數量   3819  156
       可借數量  1666  156
       可還數量  2114  156
中正區    總數量   3209  121
       可借數量  1603  121
       可還數量  1540  121
信義區    總數量   3331  106
       可借數量  1375  106
       可還數量  1842  106
內湖區    總數量   3499  168
       可借數量  1544  168
       可還數量  1902  168
北投區    總數量   2472   94
       可借數量   864   94
       可還數量  1558   94
南港區    總數量   2317   92
       可借數量   873   92
       可還數量  1435   92
士林區    總數量   2962  128
       可借數量  1318  128
       可還數量  1592  128
大同區    總數量   1630   62
       可借數量   535   62
       可還數量  1085   62
大安區    總數量   4987  185
       可借數量  2187  185
       可還數量  2742  185
文山區    總數量   2203   97
       可借數量   678   97
       可還數量  1525   97
松山區    總數量   2343   88
       可借數量   959   88
       可還數量  1321   88
臺大公館校區 總數量   1437   53
       可借數量   724   53
       可還數量   529   53
萬華區    總數量   2057   80
       可借數量   810   80
       可還數量  1235   80